In [1]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.2.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:13 http://ppa.launchpad

In [2]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2022-03-11 16:44:31--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  4.94MB/s    in 0.2s    

2022-03-11 16:44:32 (4.94 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [4]:
from pyspark import SparkFiles
# Load in Amazon Data from S3 into a DataFrame
url = 'https://benmxnguyen-bigdatahw.s3.amazonaws.com/amazon_reviews_us_Sports_v1_00.tsv'
spark.sparkContext.addFile(url)

df = spark.read.csv(SparkFiles.get('amazon_reviews_us_Sports_v1_00.tsv'), sep='\t', header=True)

In [5]:
df.show(10)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   48945260|R1WBPB8MDCCN8F|B012P7UPSM|     409940130|Chicago Blackhawk...|          Sports|          5|            0|          0|   N|                N|   LOVE IT. 6 stars!|Bought this last ...| 2015-08-31|
|         US|    5782091|R32M0YEWV77XG8|B001GQ3VHG|     657746679|Copag Poker Size ...|          Sports|          5|    

In [6]:

df.columns

['marketplace',
 'customer_id',
 'review_id',
 'product_id',
 'product_parent',
 'product_title',
 'product_category',
 'star_rating',
 'helpful_votes',
 'total_votes',
 'vine',
 'verified_purchase',
 'review_headline',
 'review_body',
 'review_date']

In [7]:
df.count()

4850360

In [8]:
df = df.drop('marketplace', 'review_headline', 'review_body', 'product_category', 'verified_purchase')

In [9]:
df.show(10)

+-----------+--------------+----------+--------------+--------------------+-----------+-------------+-----------+----+-----------+
|customer_id|     review_id|product_id|product_parent|       product_title|star_rating|helpful_votes|total_votes|vine|review_date|
+-----------+--------------+----------+--------------+--------------------+-----------+-------------+-----------+----+-----------+
|   48945260|R1WBPB8MDCCN8F|B012P7UPSM|     409940130|Chicago Blackhawk...|          5|            0|          0|   N| 2015-08-31|
|    5782091|R32M0YEWV77XG8|B001GQ3VHG|     657746679|Copag Poker Size ...|          5|            1|          1|   N| 2015-08-31|
|   45813853| RR8V7WR27NXJ5|B008VS8M58|     962442336|Baoer 223 5.56x45...|          1|            0|          0|   N| 2015-08-31|
|    1593730|R1MHO5V9Z932AY|B005F06F4U|      74305227|All Terrain Tackl...|          5|            0|          0|   N| 2015-08-31|
|   29605511|R16PD71086BD2V|B010T4IE2C|     787185588|Swim Cap - 3 Pack...|        

In [10]:
df

DataFrame[customer_id: string, review_id: string, product_id: string, product_parent: string, product_title: string, star_rating: string, helpful_votes: string, total_votes: string, vine: string, review_date: string]

In [11]:
df = df.withColumn("customer_id", df.customer_id.cast('int'))
df = df.withColumn("product_parent", df.product_parent.cast('int'))
df = df.withColumn("star_rating", df.star_rating.cast('int'))
df = df.withColumn("helpful_votes", df.helpful_votes.cast('int'))
df = df.withColumn("total_votes", df.total_votes.cast('int'))
df = df.withColumn("review_date", df.review_date.cast('date'))

In [12]:
df

DataFrame[customer_id: int, review_id: string, product_id: string, product_parent: int, product_title: string, star_rating: int, helpful_votes: int, total_votes: int, vine: string, review_date: date]

## Creating dataframes for each table

In [13]:
review_id_table_df = df[['review_id', 'customer_id', 'product_id', 'product_parent', 'review_date']]

In [14]:
review_id_table_df.show(30)

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R1WBPB8MDCCN8F|   48945260|B012P7UPSM|     409940130| 2015-08-31|
|R32M0YEWV77XG8|    5782091|B001GQ3VHG|     657746679| 2015-08-31|
| RR8V7WR27NXJ5|   45813853|B008VS8M58|     962442336| 2015-08-31|
|R1MHO5V9Z932AY|    1593730|B005F06F4U|      74305227| 2015-08-31|
|R16PD71086BD2V|   29605511|B010T4IE2C|     787185588| 2015-08-31|
|R1Z8IFGWTRWXT6|   11112959|B004RKJGLS|      94127483| 2015-08-31|
|R3AUMSHAW73HWN|     108031|B005V3DCBU|     526977496| 2015-08-31|
|R2KWDWFOHGX6FL|   13981540|B00MHT9WN8|      26521265| 2015-08-31|
|R3H9543FWBWFBU|   37993909|B001CSIRQ8|     652431165| 2015-08-31|
| RUANXOQ9W3OU5|   26040213|B001KZ3NOO|     635861713| 2015-08-31|
|R31673RTGEZSW7|   34657602|B00005RCQS|      72099763| 2015-08-31|
|R22OQLFSH42RCM|   14346192|B00FA7RWVI|     757354022| 2015-08

In [15]:
product_table_df = df[['product_id', 'product_title']]

In [16]:
product_table_df.show(10)

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B012P7UPSM|Chicago Blackhawk...|
|B001GQ3VHG|Copag Poker Size ...|
|B008VS8M58|Baoer 223 5.56x45...|
|B005F06F4U|All Terrain Tackl...|
|B010T4IE2C|Swim Cap - 3 Pack...|
|B004RKJGLS|adidas Men's Spor...|
|B005V3DCBU|Nike Men's Perfor...|
|B00MHT9WN8|Green Bay Packers...|
|B001CSIRQ8|Isokinetics Inc. ...|
|B001KZ3NOO|Aottop Quality El...|
+----------+--------------------+
only showing top 10 rows



In [17]:
customer_table_df = df['customer_id']

In [18]:
vine_table_df = df[['review_id', 'star_rating', 'helpful_votes', 'total_votes', 'vine']]

## Cleaning the dataframes

### Review ID Table DF

In [19]:
review_id_table_df

DataFrame[review_id: string, customer_id: int, product_id: string, product_parent: int, review_date: date]

In [20]:
review_id_table_df.count()

4850360

In [21]:
review_id_table_df.drop_duplicates(['review_id']).count()

4850360

### Product Table DF

In [22]:
product_table_df #Product id and product title

DataFrame[product_id: string, product_title: string]

In [23]:
product_table_df.count()

4850360

In [24]:
product_table_df = product_table_df.drop_duplicates(['product_id'])

### Customer Table DF

In [25]:
customer_table_df #customer id and customer count

Column<'customer_id'>

In [26]:
customer_table_df = df.groupby('customer_id').count().toDF('customer_id', 'customer_count')

In [27]:
customer_table_df

DataFrame[customer_id: int, customer_count: bigint]

### Vine Table DF

In [28]:
vine_table_df

DataFrame[review_id: string, star_rating: int, helpful_votes: int, total_votes: int, vine: string]

Postgres Setup

In [29]:
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://<databaseendpoint>:5432/<database>"
config = {"user":"<user>", 
          "password": "<password>", 
          "driver":"org.postgresql.Driver"}


In [ ]:
# Write DataFrame to review_id table in RDS

review_id_table_df.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [ ]:
# Write dataframe to product table in RDS

product_table_df.write.jdbc(url=jdbc_url, table='products', mode=mode, properties=config)

In [ ]:
# Write dataframe to customer table in RDS

customer_table_df.write.jdbc(url=jdbc_url, table='customers', mode=mode, properties=config)

In [ ]:
# Write dataframe to vine table in RDS

vine_table_df.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)